<a href="https://colab.research.google.com/github/baeksumin/k-project/blob/main/datacollection_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
import json
import pytz
import requests
import pandas as pd
from io import BytesIO
from datetime import datetime 
from datetime import timedelta
import pickle
from google.colab import drive

drive.mount('/content/drive')
data_path = '/content/drive/MyDrive/[Shared]K-Project/data/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
## Load pickle - fullcode 목록
with open(data_path + 'full_code.pkl',"rb") as f: 
    fullcode = pickle.load(f)
    
# 상장폐지 상태 제외시키기
fullcode_filter = fullcode[fullcode['status'] != '상장폐지']

# fullcode열을 list로 변경
fullcode_list = list(fullcode_filter['full_code'])
print(fullcode_list[:10])
print(len(fullcode_list))

['KR7060310000', 'KR7095570008', 'KR7006840003', 'KR7054620000', 'KR7265520007', 'KR7211270004', 'KR7027410000', 'KR7282330000', 'KR7138930003', 'KR7001460005']
2588


In [3]:
# 한국날짜 설정
def kor_date():
    tz = pytz.timezone('Asia/Seoul')  
    raw_dates = datetime.now(tz)
    yes = raw_dates - timedelta(1) 
    yesterday =  yes.strftime("%Y%m%d")  
    return yesterday
a = kor_date()
print(a)

20211018


In [4]:
# 데이터 가져와서 필요 컬럼 추출 후 데이터프레임 생성하여 json으로 저장하는 함수
def make_initialdata(code):
    global result
    
    data2 = { 'bld': 'dbms/MDC/STAT/issue/MDCSTAT23902',
    'isuCd': code, 
    'isuCd2': '',
    'strtDd': strtDd,
    'endDd': endDd,
    'share': '1',
    'money': '1',
    'csvxls_isNo': 'false',}

    r2 = requests.post(url2, data2, headers)
    j2 = json.loads(r2.text)
    df2 = pd.DataFrame(j2['output'])
    # print(df2)

    data3 = { 'bld': 'dbms/MDC/STAT/standard/MDCSTAT03502',
    'isuCd': code,
    'isuCd2': code,
    'strtDd': strtDd,
    'endDd': endDd,
    'searchType': '2',
    'mktId': 'ALL',
    'csvxls_isNo': 'false',}

    r2 = requests.post(url2, data3, headers)
    j2 = json.loads(r2.text)
    df3 = pd.DataFrame(j2['output'])
    # print(df3)

    if len(df2.columns) > 0: # df2에 column이 존재하면 (df가 비어서 에러나는 경우 방지)
        df2_ = df2[['TRD_DD', 'ISU_CD', 'ISU_NM', 'TDD_CLSPRC', 'TDD_OPNPRC', 'TDD_HGPRC', 'TDD_LWPRC', 'MKTCAP']]
    else: # df가 비었을때 column 생성해줌
        df2_ = pd.DataFrame(index = range(0,len(df3)),
                           columns = ['TRD_DD', 'ISU_CD', 'ISU_NM', 'TDD_CLSPRC', 'TDD_OPNPRC', 'TDD_HGPRC', 'TDD_LWPRC', 'MKTCAP'])

    if len(df3.columns) > 0:
        df3_ = df3[['EPS', 'PER', 'BPS', 'PBR', 'DPS', 'DVD_YLD']]
    else:
        df3_ = pd.DataFrame(index = range(0,len(df2)),
                           columns = ['EPS', 'PER', 'BPS', 'PBR', 'DPS', 'DVD_YLD'])
 
    result = pd.concat([df2_, df3_], axis = 1)
    # print(result)

In [5]:
with open(data_path + 'initial_data1.json',"r") as json_file:
    json_data = json.load(json_file) # json file open

In [6]:
url2 = 'http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd'
headers = {'User-Agent': 'Chrome/78.0.3904.87 Safari/537.36',}

key_list = list(json_data.keys())
result_dict = {}

for i in range(len(fullcode_list)): 

    if fullcode_list[i] in key_list: # 어제 있던 데이터가 오늘도 정상적으로 있는 경우
        df = pd.read_json(json_data[fullcode_list[i]], orient ='index') # 첫번째 키값으로 데이터프레임 변환
        json_df = df.transpose()
        
        try:
            date = json_df.iloc[0,0] # 수집된 데이터의 마지막 날짜 확인
            yesterday = re.sub('/','-',date)
            yesterday = datetime.strptime(yesterday, '%Y-%m-%d') + timedelta(days = 1)
            yesterday = yesterday.strftime("%Y%m%d")

            strtDd = yesterday # 수집된 데이터의 마지막 날짜
            endDd = kor_date() # 10월 8일 00:30 코드실행이라면 10월 7일까지의 데이터 가져와야 함

            make_initialdata(fullcode_list[i])
            
            result1 = pd.concat([result, json_df], axis = 0, ignore_index = True)

            result_js = result1.to_json(orient = 'columns') # 데이터프레임을 json 형태로 변환
            result_dict[fullcode_list[i]] = result_js # fullcode가 key값이 되고 json이 value값이 됨

            print(i, '정상')

        except:
            strtDd = 19000101
            endDd = kor_date()
            make_initialdata(fullcode_list[i])

            result_js = result.to_json(orient = 'columns') # 데이터프레임을 json 형태로 변환
            result_dict[fullcode_list[i]] = result_js # fullcode가 key값이 되고 json이 value값이 됨

            print(i, 'except문 작동', fullcode_list[i])


    elif fullcode_list[i] not in key_list: # 신규상장 된 경우
        strtDd = 19000101
        endDd = kor_date()
        make_initialdata(fullcode_list[i])

        result_js = result.to_json(orient = 'columns') # 데이터프레임을 json 형태로 변환
        result_dict[fullcode_list[i]] = result_js # fullcode가 key값이 되고 json이 value값이 됨

        print(i, '신규상장', fullcode_list[i])

    # elif key_list[i] not in fullcode_list: # 상장폐지 된 경우
    #     pass
    #     print(i, '상장폐지', key_list[i])

# 최종 결과물 저장   
with open(data_path + 'initial_data1.json', 'w') as f:
    json.dump(result_dict, f)

0 정상
1 정상
2 정상
3 정상
4 정상
5 정상
6 정상
7 정상
8 정상
9 정상
10 정상
11 정상
12 정상
13 정상
14 정상
15 정상
16 정상
17 정상
18 정상
19 정상
20 정상
21 정상
22 정상
23 정상
24 정상
25 정상
26 정상
27 정상
28 정상
29 정상
30 정상
31 정상
32 정상
33 정상
34 정상
35 정상
36 정상
37 정상
38 정상
39 정상
40 정상
41 정상
42 정상
43 정상
44 정상
45 정상
46 정상
47 정상
48 정상
49 정상
50 정상
51 정상
52 정상
53 정상
54 정상
55 정상
56 정상
57 정상
58 정상
59 정상
60 정상
61 정상
62 정상
63 정상
64 정상
65 정상
66 정상
67 정상
68 정상
69 정상
70 정상
71 정상
72 정상
73 정상
74 정상
75 정상
76 정상
77 정상
78 정상
79 정상
80 정상
81 정상
82 정상
83 정상
84 정상
85 정상
86 정상
87 정상
88 정상
89 정상
90 정상
91 정상
92 정상
93 정상
94 정상
95 정상
96 정상
97 정상
98 정상
99 정상
100 정상
101 정상
102 정상
103 정상
104 정상
105 정상
106 정상
107 정상
108 정상
109 정상
110 정상
111 정상
112 정상
113 정상
114 정상
115 정상
116 정상
117 정상
118 정상
119 정상
120 정상
121 정상
122 정상
123 정상
124 정상
125 정상
126 정상
127 정상
128 정상
129 정상
130 정상
131 정상
132 정상
133 정상
134 정상
135 정상
136 정상
137 정상
138 정상
139 정상
140 정상
141 정상
142 정상
143 정상
144 정상
145 정상
146 정상
147 정상
148 정상
149 정상
150 정상
151 정상
152 정상
153 정상
154 정상
155 정상
156 정상
157 정상
158 

In [7]:
# 에러 발생 상황
# 신규상장되어 json data에 데이터가 없는 상황 --> if-else문으로 처리
# json data에 fullcode가 있는데 데이터프레임이 비어 있는 상황 --> try-except문으로 처리

In [8]:
# 저장된 데이터 가져와서 정상적으로 저장되었는지 테스트
with open(data_path + 'initial_data1.json',"r") as json_file:
    json_data = json.load(json_file) # json file open

df = pd.read_json(json_data['KR7060310000'], orient ='index') # 첫번째 키값으로 데이터프레임 변환
json_df = df.transpose()
print(json_df)

          TRD_DD  ISU_CD  ISU_NM TDD_CLSPRC  ...  BPS   PBR DPS DVD_YLD
0     2021/10/18  060310      3S      4,590  ...  792  5.80   0    0.00
1     2021/10/15  060310      3S      4,065  ...  792  5.13   0    0.00
2     2021/10/14  060310      3S      3,995  ...  792  5.04   0    0.00
3     2021/10/13  060310      3S      3,790  ...  792  4.79   0    0.00
4     2021/10/12  060310      3S      3,745  ...  792  4.73   0    0.00
...          ...     ...     ...        ...  ...  ...   ...  ..     ...
4813  2002/04/29  060310  삼에스코리아      6,780  ...    -     -   -       -
4814  2002/04/26  060310  삼에스코리아      7,700  ...    -     -   -       -
4815  2002/04/25  060310  삼에스코리아      8,750  ...    -     -   -       -
4816  2002/04/24  060310  삼에스코리아      8,510  ...    -     -   -       -
4817  2002/04/23  060310  삼에스코리아      7,600  ...    -     -   -       -

[4818 rows x 14 columns]


In [8]:
# 데이터가 정상적으로 들어간 것 확인!